# Alzheimer's disease Drug Repurposing via disease-compounds relations

这个例子展示了如何使用我们的预训练模型 (TransE_l1)进行药物重定位.

## Collecting Alzheimer's disease

一开始我们需要收集 DRKG 中的 AD 列表. 我们能够使用 DRKG 中的疾病 ID 编码疾病. 下面我们将全部的 AD 疾病作为目标. 

In [1]:
AD_disease_list = [
'Disease::DOID:10652',
'Disease::MESH:C536599',
'Disease::MESH:D000544'
]

In [2]:
len(AD_disease_list)

3

In [3]:
AD_disease_list

['Disease::DOID:10652', 'Disease::MESH:C536599', 'Disease::MESH:D000544']

## Candidate drugs

现在我们使用 Drugbank 中的 FDA 批准的药物作为候选药物.（我们排除分子量 < 250 的药物）药物清单在 infer\_drug.tsv 中.

In [ ]:
import csv

# Load entity file
drug_list = []
with open("D:/........./filtered_infer_drug.tsv", newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t', fieldnames=['drug','ids'])
    for row_val in reader:
        drug_list.append(row_val['drug'])

In [5]:
len(drug_list)

7926

In [6]:
drug_list[:3]

['Compound::DB00605', 'Compound::DB00983', 'Compound::DB01240']

## Treatment relation

In [7]:
treatment_list = [
'DRUGBANK::treats::Compound:Disease',
'GNBR::T::Compound:Disease',
'Hetionet::CtD::Compound:Disease'
]

In [8]:
treatment_list

['DRUGBANK::treats::Compound:Disease',
 'GNBR::T::Compound:Disease',
 'Hetionet::CtD::Compound:Disease']

## DRKG 中 AD 的治疗药物

In [ ]:
ad_drugs = []

with open("D:/........./filtered_ad_drugs.tsv", encoding='utf-8') as f:
    for drug in f:
        ad_drugs.append(drug[:-1])

In [10]:
len(ad_drugs)

126

In [11]:
ad_drugs

['Compound::DB13588',
 'Compound::DB09130',
 'Compound::DB00244',
 'Compound::DB04815',
 'Compound::DB03994',
 'Compound::DB00712',
 'Compound::DB00158',
 'Compound::DB00472',
 'Compound::DB00331',
 'Compound::DB01593',
 'Compound::DB11100',
 'Compound::DB03128',
 'Compound::DB06712',
 'Compound::DB00993',
 'Compound::DB00877',
 'Compound::DB11094',
 'Compound::DB04115',
 'Compound::DB11805',
 'Compound::DB00843',
 'Compound::DB00928',
 'Compound::DB11068',
 'Compound::DB00694',
 'Compound::DB11748',
 'Compound::DB14500',
 'Compound::DB00382',
 'Compound::DB00393',
 'Compound::DB00490',
 'Compound::DB00215',
 'Compound::DB03843',
 'Compound::DB06756',
 'Compound::DB05289',
 'Compound::DB00564',
 'Compound::DB05381',
 'Compound::DB01235',
 'Compound::DB11780',
 'Compound::DB00945',
 'Compound::DB00571',
 'Compound::DB00674',
 'Compound::DB01234',
 'Compound::DB00134',
 'Compound::DB08842',
 'Compound::DB00136',
 'Compound::DB00368',
 'Compound::DB12310',
 'Compound::DB00682',
 'Compound

## Get pretrained model

我们能直接使用我们的预训练模型 (TransE_l1)做药物重定位.

In [ ]:
entity_id_file = 'D:/........./entities.tsv'
relation_id_file = 'D:/........./relations.tsv'

## Get embeddings for diseases and drugs

In [13]:
# Get drugname/disease name to entity ID mappings
entity_map = {}
entity_id_map = {}
relation_map = {}
with open(entity_id_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t', fieldnames=['id', 'name'])
    for row_val in reader:
        entity_map[row_val['name']] = int(row_val['id'])
        entity_id_map[int(row_val['id'])] = row_val['name']
        
with open(relation_id_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t', fieldnames=['id', 'name'])
    for row_val in reader:
        relation_map[row_val['name']] = int(row_val['id'])
        
# handle the ID mapping
drug_ids = []
disease_ids = []
for drug in drug_list:
    drug_ids.append(entity_map[drug])
    
for disease in AD_disease_list:
    disease_ids.append(entity_map[disease])

treatment_ids = [relation_map[treat]  for treat in treatment_list]

In [14]:
len(disease_ids),len(drug_ids),len(treatment_ids)

(3, 7926, 3)

In [15]:
disease_ids, drug_ids[:3], treatment_ids

([4018, 30136, 5882], [6346, 6506, 1391], [99, 98, 100])

In [ ]:
# Load embeddings
import torch
import numpy as np
entity_emb = np.load('D:/........./entity_embedding_LLaDR.npy')
rel_emb = np.load('D:/........./relation_embedding_LLaDR.npy')

drug_ids = torch.tensor(drug_ids).long()
disease_ids = torch.tensor(disease_ids).long()
treatment_ids = torch.tensor(treatment_ids)

drug_emb = torch.tensor(entity_emb[drug_ids])
treatment_embs = [torch.tensor(rel_emb[r_id]) for r_id in treatment_ids]

In [17]:
disease_ids, drug_ids[:3], treatment_ids

(tensor([ 4018, 30136,  5882]),
 tensor([6346, 6506, 1391]),
 tensor([ 99,  98, 100]))

In [18]:
drug_emb.shape

torch.Size([7926, 400])

## Drug Repurposing Based on Edge Score

我们使用下面算法来计算 the edge score. 注意, 这里我们用 logsigmoid 函数使全部 scores < 0. 分数越大, $(h, r, t)$ 越可能成立.

$\mathbf{d} = \gamma - ||\mathbf{h}+\mathbf{r}-\mathbf{t}||_{1}$

$\mathbf{score} = \log\left(\frac{1}{1+\exp(\mathbf{-d})}\right)$

在进行药物重定位时，我们只使用与治疗相关的关系.

DGL-KE 官方实现 TransE_l1 评分函数的代码在:

- https://github.com/awslabs/dgl-ke/blob/master/python/dglke/models/ke_model.py 886 - 893 行

- https://github.com/awslabs/dgl-ke/blob/master/python/dglke/models/pytorch/score_fun.py 54 - 59 行

OpenKE 实现 TransE_l1 评分函数的代码在:

- https://github.com/thunlp/OpenKE/blob/OpenKE-PyTorch/openke/module/model/TransE.py

其中两者的实现代码基本一样.

In [19]:
import torch.nn.functional as fn

gamma = 18.0
def distMult(head, rel, tail):  
    # 计算评分，使用点积  
    score = torch.matmul(head, rel) * torch.matmul(rel, tail)  # 任意数量的元素  
    return score  
def transel1(head_embs, relation_embs, tail_embs, gamma=18.0):
    """Vectorized scoring function with potential GPU acceleration"""
    scores = gamma - torch.norm(head_embs + relation_embs - tail_embs, p=1, dim=1)
    return scores
scores_per_disease = []
drugs = []
for r_id in range(len(treatment_embs)):
    gamma=18.0
    treatment_emb = treatment_embs[r_id]
    for disease_id in disease_ids:
        disease_emb = entity_emb[disease_id]
        score = fn.logsigmoid(transel1(drug_emb, treatment_emb, disease_emb,gamma))
        scores_per_disease.append(score)
        drugs.append(drug_ids)
scores = torch.cat(scores_per_disease)
drugs = torch.cat(drugs)

In [20]:
scores.shape, drugs.shape, 3*3*8104

(torch.Size([71334]), torch.Size([71334]), 72936)

In [21]:
# sort scores in decending order
# torch.flip: Reverse the order of a n-D tensor along given axis in dims.
idx = torch.flip(torch.argsort(scores), dims=[0])
scores = scores[idx].numpy()
drugs = drugs[idx].numpy()

scores.shape, drugs.shape, 3*3*8104

((71334,), (71334,), 72936)

### Now we output proposed treatments

In [22]:
_, unique_indices = np.unique(drugs, return_index=True)
topk = 50
topk_indices = np.sort(unique_indices)[:topk]
proposed_drugs = drugs[topk_indices]
proposed_scores = scores[topk_indices]
true_ad_drugs = set(ad_drugs)

In [23]:
top50_list = []

for i in range(topk):
    drug = int(proposed_drugs[i])
    if entity_id_map[drug] in ad_drugs:
        score = proposed_scores[i]
        row = "[{}],{},{}\n".format(i+1, entity_id_map[drug], score)
        top50_list.append(row)
        print("[{}]\t{}\t{}".format(i+1, entity_id_map[drug], score))

[28]	Compound::DB01698	-8.955840894486755e-05
